<a href="https://colab.research.google.com/github/Yolantele/ML-data-clasifier/blob/master/SpaCy_ML_Classifier_for_Waste_Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####**SpaCy Data Classification POC**

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# mount data from drive
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/data/'


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install -U spacy==2.2.2
!pip install pandas

In [ ]:
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import pandas as pd
from spacy.lang.en import English
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

# References:
# https://www.dataquest.io/blog/tutorial-text-classification-in-python-using-spacy/

spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")

In [ ]:
materials = pd.read_csv(path + '/enMaterialData.csv')
# or use test data frame where material field is empty
materials_test = pd.read_csv(path + '/enWithoutMaterialData.csv')

df = materials
# df.head()
# df.info()
# df.description + df.euralDescription
df.description + df.euralDescription


0       Soyadroes technicallymaterial unsuitable for c...
1                             Brancheswaste from forestry
2       Cocoa shellsmaterial unsuitable for consumptio...
3       Debris with Sandmixtures of concrete, stones, ...
4       soyamaterial unsuitable for consumption or pro...
                              ...                        
2115                                  LDPE - 98:2plastics
2116    hollow variegated glass, route collection fire...
2117    Soil mixed with contaminated stonessoil and st...
2118    Stainless steel, Motors, Refiner, Tin / Lead, ...
2119    soil / gravel / stones / cloths oil contaminat...
Length: 2120, dtype: object

###Tokening the Data With spaCy

Now that we know what we’re working with, let’s create a custom tokenizer function using spaCy. We’ll use this function to automatically strip information we don’t need, like stopwords and punctuation, from each review.

In [ ]:

string = "mixtures of concrete, stones, tiles or ceramic products other than those mentioned in 17 01 06"
string2 = 'B and O land class LIVING - B'

punctuations = '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~'
stop_words = spacy.lang.en.stop_words.STOP_WORDS

def spacy_tokenizes(docs, logging=False):
    texts = []
    doc = nlp(docs, disable=['parser', 'ner'])
    tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
    tokens = [tok for tok in tokens if tok not in punctuations  and tok not in stop_words ]
    return tokens

tokens = spacy_tokenizes(string2)
print(tokens)


['b', 'o', 'land', 'class', 'living', 'b']


In [ ]:
# Different tokenizer (leaves words in plurals)
parser = English()

def spacy_tokenize(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    sentence = sentence.strip().lower()
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

tokens = spacy_tokenize(string2)
print(tokens)

['b', 'o', 'land', 'class', 'living', 'b']


###Vectorization Feature Engineering (TF-IDF) , Bag of Words and N-grams

Classifying text we end up with text snippets with their respective labels. But in machine learning model we need to convert into numeric representation

TF-IDF -Term Frequency-Inverse Document Frequency - simply a way of normalizing our Bag of Words(BoW) by looking at each word’s frequency in comparison to the document frequency.

N-grams - combinations of adjacent words in a given text. For example "who will win"
- when n = 1, becomes "who", "will", "win"
- when n = 2 , becomes "who will", "will win" etc. 

In [ ]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

print(bow_vector)

NameError: ignored

In [ ]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)